In [2]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.6 MB 2.5 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.6 MB 3.7 MB/s eta 0:00:03
   -------------- ------------------------- 3.4/9.6 MB 4.2 MB/s eta 0:00:02
   ---------------------- ----------------- 5.5/9.6 MB 5.4 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.6 MB 5.9 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.6 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.6 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      

In [3]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [5]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [6]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

## Question 1: Use yfinance to Extract Stock Data

In [ ]:
tesla = yf.Ticker("TSLA")

In [8]:
tesla_data = tesla.history(period="max")

In [9]:
tesla_data.reset_index(inplace=True)

print("\nFirst five rows of tesla_data:")
print(tesla_data.head())


First five rows of tesla_data:
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


## Question 2: Use Webscraping to Extract Tesla Revenue Data

In [10]:
import requests
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text

In [11]:
soup = BeautifulSoup(html_data, 'html.parser')

In [13]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

response = requests.get(url)

html_data = response.text

soup = BeautifulSoup(html_data, 'html.parser')

tables = soup.find_all("table")
target_table = None
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        target_table = table
        break

if target_table:
    for row in target_table.find("tbody").find_all("tr"):
        col = row.find_all("td")
        if len(col) >= 2:  
            date = col[0].text.strip()  
            revenue = col[1].text.strip()  
            
            tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({
                "Date": [date],
                "Revenue": [revenue]
            })], ignore_index=True)

In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"",regex=True)

In [15]:
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [16]:
print("Last five rows of tesla_revenue:")
print(tesla_revenue.tail())

Last five rows of tesla_revenue:
          Date Revenue
48  2010-09-30      31
49  2010-06-30      28
50  2010-03-31      21
52  2009-09-30      46
53  2009-06-30      27


## Question 3: Use yfinance to Extract Stock Data

In [17]:
import yfinance as yf
gamestop = yf.Ticker("GME")

In [18]:
gme_data = gamestop.history(period="max")

In [19]:
gme_data.reset_index(inplace=True)

print("First five rows of gme_data:")
print(gme_data.head())

First five rows of gme_data:
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620129  1.693350  1.603296  1.691667  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


## Question 4: Use Webscraping to Extract GME Revenue Data

In [20]:
import requests
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
response = requests.get(url)
html_data_2 = response.text

In [21]:
soup = BeautifulSoup(html_data_2, 'html.parser')

In [22]:
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
target_table = soup.find_all("tbody")[1]

for row in target_table.find_all("tr"):
    col = row.find_all("td")
    if len(col) >= 2:  
        date = col[0].text.strip()  
        revenue = col[1].text.strip().replace('$', '').replace(',', '')  
        
        gme_revenue = pd.concat([gme_revenue, pd.DataFrame({
            "Date": [date],
            "Revenue": [revenue]
        })], ignore_index=True)

In [23]:
print("Last five rows of gme_revenue:")
print(gme_revenue.tail())

Last five rows of gme_revenue:
          Date Revenue
57  2006-01-31    1667
58  2005-10-31     534
59  2005-07-31     416
60  2005-04-30     475
61  2005-01-31     709


## Question 5: Plot Tesla Stock Graph

In [29]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

tesla_data.reset_index(inplace=True)

tesla_data['Date'] = pd.to_datetime(tesla_data['Date'])
tesla_data = tesla_data[tesla_data['Date'] <= '2021-06-30']

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text
soup = BeautifulSoup(html_data, 'html.parser')
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

def make_graph(stock_data, revenue_data, stock_name):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(
        go.Scatter(x=stock_data['Date'], y=stock_data['Close'], name=f"{stock_name} Stock Price"),
        secondary_y=False
    )
    
    fig.add_trace(
        go.Scatter(x=revenue_data['Date'], y=revenue_data['Revenue'], name=f"{stock_name} Revenue"),
        secondary_y=True
    )
    
    fig.update_layout(
        title=f"{stock_name} Stock Price and Revenue (Up to June 2021)",
        xaxis_title="Date",
        template="plotly_white"
    )
    fig.update_yaxes(title_text="Stock Price ($)", secondary_y=False)
    fig.update_yaxes(title_text="Revenue ($M)", secondary_y=True)
    
    fig.show()

make_graph(tesla_data, tesla_revenue, 'Tesla')

## Question 6: Plot GameStop Stock Graph

In [31]:
gme_data['Date'] = pd.to_datetime(gme_data['Date'])
gme_data = gme_data[gme_data['Date'] <= '2021-06-30']
make_graph(gme_data, gme_revenue, 'GameStop')